# MNIST classification using persistent homology and vectorizations

In this notebook, we apply TDA to the MNIST dataset consisting of handwritten digits for classification. This notebook borrows heavily from the `giotto-tda` tutorial notebook [Classifying handwritten digits](https://giotto-ai.github.io/gtda-docs/0.4.0/notebooks/MNIST_classification.html), except for some slight modifications in the beginning and the end. I strongly encourage you to check out all that `giottto-tda` has to offer, and especially read the [giotto-tda documentation](https://giotto-ai.github.io/gtda-docs/0.4.0/index.html).

The MNIST dataset consists of 70,000 handwritten digits, stored as grey scale images of 28 x 28 pixels. We will build a classifier to predict which digit is written from the image. See [The MNIST Database of Handwritten Digits](http://yann.lecun.com/exdb/mnist/) by Yann LeCun, Corinna Cortes, and Christopher J.C. Burges for more details. 

For a more in-depth version of the basic pipeline outlined here, see [A Topological "Reading" Lesson: Classification of MNIST using TDA](https://arxiv.org/abs/1910.08345) by Adélie Garin and Guillaume Tauzin.

### Load the dataset, binarize, split into training and test sets

We load the data set using `sklearn` and cast the targets as integers (instead of strings).

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target']
y = y.astype(np.uint8)

In [ ]:
X.shape, y.shape

So we do indeed have 70,000 images, each of which has 784 = 28 * 28 pixels. Let's check out a digit to make sure things work the way we think.

In [ ]:
import matplotlib.pyplot as plt
digit = X[4].reshape(28,28)

plt.imshow(digit, cmap = 'binary')
plt.axis('off')

That looks like a 9. Is it?

In [ ]:
y[4]

We need to pick a threshold for the Binarizer, so if the pixel value is greater than threshold, the pixel is on. Otherwise it is off. We choose 0.4, the same threshold as the [Garin et al] paper.

In [ ]:
from sklearn.preprocessing import Binarizer
binarizer = Binarizer(threshold=1)
digit_bin = binarizer.fit_transform(digit)

plt.imshow(digit_bin, cmap='binary')
plt.axis('off')
plt.savefig('thresh1')

Finally, we're ready to split the data into a training set and a test set. The `train_size` and `test_size` parameters below can be increased to make a better model at the cost of computation time.

In [ ]:
from sklearn.model_selection import train_test_split

train_size, test_size = 600, 100

# Reshape to (n_samples, n_pixels_x, n_pixels_y)
X = X.reshape((-1, 28, 28))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_size, test_size=test_size, stratify=y, random_state=42
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

### Pipeline with a single digit

The following pipeline can be a bit much if you're new to data science and TDA, so lets step through the entire pipeline using our binarized digit `digit`. We will think of the digit as a _cubical complex_ $C$ and build a filtration of the digit using various filtrations discussed earlier. 

The _height filtration_ is determined by a _height_ function $h : C \times S^{1} \to \mathbb{R}$, assigning to each voxel, direction pair $(c,\theta)$ the distance from the first seen edge (hyperplane) of $C$ in the direction of $\theta$ to $c$. Typically we fix the direction $\theta$ and use $h_{\theta} : C \to \mathbb{R}$ to construct the filtration. If $v$ is a unit vector pointing in the direction of $\theta$, then $h_{\theta}(c) = \langle c ,v \rangle$. Thus $\mathcal{F}_{\theta}(r) = \{ c | h(c,\theta) \leq r \}$.

The _radial filtration_ is determined by a _radial_ function $r : C \times C \to \mathbb{R}$, assigning to each voxel, 'center' pairs $(c,\rho)$ the distance from $c$ to $\rho$. Again, this is a function of the 'center' point $\rho$, which we fix and use $r_{\rho} : C \to \mathbb{R}$. If a voxel $v$ is not in the binarized image, we set $r_{\rho}(v) = r_{\infty}$, the maximum value of $r_{\rho}$. Note that this function also depends on how we measure 'distance', but for this code, we'll use either the $L_1$ or $L_2$ norm of the difference.

Sticking closely to the [Garin et. al.], we pick a uniform set of directions and centers for the height filtrations, shown in the figure below.
<img src="data/directions_and_centers.png" width="200" height="200" />

Let's try these out for the binarized digit we've been working with.

In [ ]:
from gtda.images import RadialFiltration

radial_filtration = RadialFiltration(center=np.array([5,25]))
digit_rad_filtration = radial_filtration.fit_transform_plot(digit_bin[None,:,:],colorscale='turbo')

In [ ]:
from gtda.images import HeightFiltration

height_filtration = HeightFiltration(direction=np.array([1,0]))
digit_height_filtration = height_filtration.fit_transform_plot(digit_bin[None,:,:], colorscale='turbo')

The next step is to use either of these filtrations to compute the corresponding persistent homology. Let's use the height filtration as an example.

In [ ]:
from gtda.homology import CubicalPersistence

cubical_persistence = CubicalPersistence(n_jobs=-1)
digit_height_persistence = cubical_persistence.fit_transform_plot(digit_height_filtration)

Now it's time for vectorization. We have several choices of vectorization schemes to use: persistence landscapes, heat kernels, betti curves, etc. We will ultimately use all of these in our pipeline, but for the single digit analysis we'll use the heat kernel.

In [ ]:
from gtda.diagrams import HeatKernel

heat = HeatKernel(sigma=.15, n_bins=60, n_jobs=-1)
digit_heat = heat.fit_transform(digit_height_persistence)

# Visualise the heat kernel for H1
heat.plot(digit_heat, homology_dimension_idx=1, colorscale='turbo')

The final step in our pipeline is to take the 'Amplitude' or $L_p$ norm of each vectorization in each homological degree. The Amplitude is a vector $\vec{a}=(a_0,a_1,\ldots)$ where each $a_i$ is the $L_p$ norm of the chosen vectorization scheme in homological degree $i$.

### Pipeline with the full training set

We combine all of the different filtrations, vectorization schemes, and amplitudes to make one final pipeline. This specific pipeline is taken from [Classifying handwritten digits](https://giotto-ai.github.io/gtda-docs/0.4.0/notebooks/MNIST_classification.html). 

In [ ]:
from sklearn.pipeline import make_pipeline, make_union
from gtda.diagrams import PersistenceEntropy, Scaler, Amplitude
from gtda.images import HeightFiltration, Binarizer


direction_list = [[1, 0], [1, 1], [0, 1], [-1, 1], [-1, 0], [-1, -1], [0, -1], [1, -1]]

center_list = [
    [13, 6],
    [6, 13],
    [13, 13],
    [20, 13],
    [13, 20],
    [6, 6],
    [6, 20],
    [20, 6],
    [20, 20],
]

# Creating a list of all filtration transformer, we will be applying
filtration_list = (
    [
        HeightFiltration(direction=np.array(direction), n_jobs=-1)
        for direction in direction_list
    ]
    + [RadialFiltration(center=np.array(center), n_jobs=-1) for center in center_list]
)

# Creating the diagram generation pipeline
diagram_steps = [
    [
        Binarizer(threshold=0.4),
        filtration,
        CubicalPersistence(n_jobs=-1),
        Scaler(n_jobs=-1),
    ]
    for filtration in filtration_list
]

# Listing all metrics we want to use to extract diagram amplitudes
metric_list = [
    {"metric": "bottleneck", "metric_params": {}},
    {"metric": "wasserstein", "metric_params": {"p": 1}},
    {"metric": "wasserstein", "metric_params": {"p": 2}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 2, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 2, "n_bins": 100}},
    {"metric": "betti", "metric_params": {"p": 1, "n_bins": 100}},
    {"metric": "betti", "metric_params": {"p": 2, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 1.6, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 3.2, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 1.6, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 3.2, "n_bins": 100}},
]

#
feature_union = make_union(
    *[PersistenceEntropy(nan_fill_value=-1)]
    + [Amplitude(**metric, n_jobs=-1) for metric in metric_list]
)

tda_union = make_union(
    *[make_pipeline(*diagram_step, feature_union) for diagram_step in diagram_steps],
    n_jobs=-1
)

It can be helpful to visualize the entire Pipeline using sklearn.

In [ ]:
from sklearn import set_config
set_config(display='diagram')  

tda_union

In [ ]:
X_train.shape

In [ ]:
X_train_tda = tda_union.fit_transform(X_train)
X_train_tda.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train_tda, y_train)

X_test_tda = tda_union.transform(X_test)
rf.score(X_test_tda, y_test)

How can we improve this classification? We can try to reduce some of the 476 topological features used here. By loooking at the correlation between the features, we can remove highly correlated features and reduce the complexity of the model, also improving efficiency. We could also try other ML algorithms besides Random Forests like SVM, clustering, etc. Even easier, we can increase the training set size to 6,000 or even 60,000.